In [ ]:
if TRACES != 'SIMULATED':
    scope.adc.bits_per_sample = 8

In [1]:
import random
import logging
import time
import numpy as np
from tqdm.notebook import tnrange
from ecpy.curves import Curve, Point
curve = Curve.get_curve('NIST-P256')

In [2]:
def random_k(bits=256, tries=100):
    import random
    for i in range(tries):
        k = random.getrandbits(bits)
        if k < curve.order and k > 0:
            return k
    raise ValueError("Failed to generate a valid random k after %d tries!" % self.tries)

def regularized_k(input_k, bits=256):
    """Given input k, return the regularized k that the target processes (which the attack will retrieve).
    """
    assert input_k < curve.order
    kr = input_k + curve.order
    if kr & 2**bits:
        kr -= 2**bits
    return kr   

def input_k(kr, bits=256):
    """Given the regularized k that the target processes (which the attack will retrieve), return the regularized k that the target will be processing.
    """
    if kr < curve.order:
        kr += 2**bits
    i_k = kr - curve.order
    assert i_k < curve.order # sanity check
    return i_k

In [89]:
def new_point():
    tries = 100
    for i in range(tries):
        x = random.getrandbits(256)
        y = curve.y_recover(x)
        if y:
            return (x,y)
    raise ValueError('Failed to generate a random point')



In [ ]:
def get_ttimes(step=None):
    if TRACES == 'SIMULATED':
        raws = np.load('data/uecc_ttimes%s.npz' % step, allow_pickle=True)
        ttimes = raws['arr_0']
        raws.close()
    else:
        ttimes = scope.trigger.get_trigger_times()
        if TRACES == 'COLLECT' and step:
            np.savez_compressed('data/uecc_ttimes%s.npz' % step, np.asarray(ttimes, dtype=np.uint16))
    return ttimes

In [ ]:
def capture_ecc_traces(k, N=1, check_sad_triggers=False, check_ttimes=False, Px=None, Py=None, operation="pmult", check=True, get_trace=True, as_int=True, step=1, trim=False):
    from chipwhisperer.common.traces import Trace
    traces = []
    if TRACES == 'SIMULATED':
        # eh maybe not optimal but it works
        raws = np.load('data/uecc_%s.npz' % step, allow_pickle=True)
        for t in raws['arr_0']:
            traces.append(Trace(t[0], t[1], t[2], None))
        raws.close()
        print('Pre-recorded traces loaded ✅.')
    
    else:
        if Px:
            fixed_point = True
        else:
            fixed_point = False
        for i in tnrange(N, desc='Capturing traces'):
            if not fixed_point:
                Px, Py = new_point()

            start_cycles = scope.adc.trig_count
            scope.arm()

            if operation == 'pmult':
                textout = run_pmult(k, Px, Py, check=check, verbose=False)
            else:
                logging.error("Please supply a valid operation to run.")

            ret = scope.capture()
            cycles = scope.adc.trig_count - start_cycles

            if ret:
                logging.warning("Timeout happened during capture")
                next

            textin = {'operation': operation,
                      'Px': Px,
                      'Py': Py,
                      'k': k
                      }
            textout['cycles'] = cycles
            if get_trace:
                wave = scope.get_last_trace(as_int=as_int)
                if trim:
                    assert scope.adc.segments > 1, 'trim feature is intended to be used with segmented capture'
                    trimmed_wave = np.zeros(scope.adc.segments*trim, dtype=np.uint8)
                    #print('XXX %d' % len(trimmed_wave))
                    for s in range(scope.adc.segments):
                        trimmed_wave[s*trim:(s+1)*trim] = wave[s*scope.adc.samples:s*scope.adc.samples+trim]
                    wave = trimmed_wave
                    #print('XXX %d' % len(trimmed_wave))
                    #print('XXX %d' % len(wave))
                skip = False
                if check_sad_triggers:
                    if scope.SAD.num_triggers_seen not in [255,256]:
                        print('bad trace: skipping because scope.SAD.num_triggers_seen = %d' % scope.SAD.num_triggers_seen)
                        skip = True
                if not skip and check_ttimes:
                    ttimes = scope.trigger.get_trigger_times()
                    if max(ttimes)/min(ttimes) > 1.15:
                        print('bad trace: min(ttimes)=%d, max(ttimes)=%d' % (min(ttimes), max(ttimes)))
                        skip = True
                    textout['ttimes'] = ttimes
                if not skip:
                    traces.append(Trace(wave, textin, textout, None))

        if TRACES == 'COLLECT':
            np.savez_compressed('data/uecc_%s.npz' % step, np.asarray(traces, dtype=object))
    
    return traces

In [ ]:
def capture_ecc_trace(k, Px=None, Py=None, operation="pmult", check=True, get_trace=True, as_int=True, step=1):
    from chipwhisperer.common.traces import Trace
    
    if TRACES == 'SIMULATED':
        raise ValueError('Not available.')
    
    else:
        start_cycles = scope.adc.trig_count
        scope.arm()

        if operation == 'pmult':
            textout = run_pmult(k, Px, Py, check=check, verbose=False)
        else:
            logging.error("Please supply a valid operation to run.")

        ret = scope.capture()
        cycles = scope.adc.trig_count - start_cycles

        if ret:
            logging.warning("Timeout happened during capture")
            return None

        textin = {'operation': operation,
                  'Px': Px,
                  'Py': Py,
                  'k': k
                  }
        textout['cycles'] = cycles
        if get_trace:
            wave = scope.get_last_trace(as_int=as_int)
            trace = Trace(wave, textin, textout, None)
        else:
            trace = None
    
    return trace

In [ ]:
def run_pmult(k, Px=None, Py=None, check=True, verbose=False):
    """Run an arbitrary pmult.
    Args:
        Px (int): X coordinate of curve point
        Py (int): Y coordinate of curve point
        k (int): multiplier
        check: if set, verify the result (using ecpy)
    """
    from ecpy.curves import Curve,Point
    curve = Curve.get_curve('NIST-P256')
    if Px == None:
        op = 'f'
        Px = curve.generator.x
        Py = curve.generator.y
    else:
        op = 'k'
        target.simpleserial_write('a', bytearray(int.to_bytes(Px, 32, byteorder='big')))
        time.sleep(0.1)
        target.simpleserial_write('b', bytearray(int.to_bytes(Py, 32, byteorder='big')))
        time.sleep(0.1)

    target.simpleserial_write(op, bytearray(int.to_bytes(k, 32, byteorder='big')))

    if not target.is_done():
        logging.warning ("Target not done yet, increase clksleeptime!")
        #let's wait a bit more, see what happens:
        i = 0
        while not target.is_done():
            i += 1
            time.sleep(0.05)
            if i > 100:
                logging.warning("Target still did not finish operation!")
                break

    time.sleep(1)

    target.simpleserial_write('p', bytes([0]*32))
    Rx = target.simpleserial_read('r', 32)

    target.simpleserial_write('q', bytes([0]*32))
    Ry = target.simpleserial_read('r', 32)

    Rx = int.from_bytes(Rx, byteorder='big')
    Ry = int.from_bytes(Ry, byteorder='big')
    
    # optionally check result:
    if check:
        P = Point(Px, Py, curve)
        Q = k*P
        if verbose:
            print("Expecting Qx = %s" % hex(Q.x))
            print("Expecting Qy = %s" % hex(Q.y))
        if Q.x != Rx:
            print("Bad Rx!")
            print("expected %s" % hex(Q.x))
            print("got      %s" % hex(Rx))
        if Q.y != Ry:
            print("Bad Ry!")
            print("expected %s" % hex(Q.y))
            print("got      %s" % hex(Ry))
    return {'Rx': Rx, 'Ry': Ry}

In [ ]:
def rerun_pmult(k, verbose=False):
    """Re-run an arbitrary pmult, using the same point that was used previously.
    Args:
        k (int): multiplier
    """
    op = 'k'
    trace.simpleserial_write(op, bytearray(int.to_bytes(k, 32, byteorder='big')))

    if not target.is_done():
        logging.warning ("Target not done yet, increase clksleeptime!")
        #let's wait a bit more, see what happens:
        i = 0
        while not target.is_done():
            i += 1
            time.sleep(0.05)
            if i > 100:
                logging.warning("Target still did not finish operation!")
                break
    return

In [ ]:
def capture_ecc_trace_cwlite(k, Px=None, Py=None, operation="pmult", check=True, verbose=False, cycles=6000000):
    from chipwhisperer.common.traces import Trace
    from tqdm import tnrange
    import math
    import numpy as np
    start_cycles = scope.adc.trig_count

    if operation == 'pmult':
        from ecpy.curves import Curve,Point
        curve = Curve.get_curve('NIST-P256')
        if Px == None:
            op = 'f'
            Px = curve.generator.x
            Py = curve.generator.y
        else:
            op = 'k'
            target.simpleserial_write('a', bytearray(int.to_bytes(Px, 32, byteorder='big')))
            time.sleep(0.1)
            target.simpleserial_write('b', bytearray(int.to_bytes(Py, 32, byteorder='big')))
            time.sleep(0.1)

        segments = math.ceil(cycles / scope.adc.samples)
        scope.adc.offset = 0
        wave = np.array([])
        for j in tnrange(segments, desc='Capturing trace segments'):
            scope.arm()
            trace.simpleserial_write(op, bytearray(int.to_bytes(k, 32, byteorder='big')))
            #wavesegment = scope.get_last_trace()
            while not target.is_done():
                pass
            time.sleep(1)
            ret = scope.capture()
            if ret:
                print("Failed capture")
                continue
            wave = np.append(wave, scope.get_last_trace())
            scope.adc.offset += scope.adc.samples

        time.sleep(1)

        target.simpleserial_write('p', bytes([0]*32))
        Rx = target.simpleserial_read('r', 32)

        target.simpleserial_write('q', bytes([0]*32))
        Ry = target.simpleserial_read('r', 32)

        Rx = int.from_bytes(Rx, byteorder='big')
        Ry = int.from_bytes(Ry, byteorder='big')

        # optionally check result:
        if check:
            P = Point(Px, Py, curve)
            Q = k*P
            if verbose:
                print("Expecting Qx = %s" % hex(Q.x))
                print("Expecting Qy = %s" % hex(Q.y))
            if Q.x != Rx:
                print("Bad Rx!")
                print("expected %s" % hex(Q.x))
                print("got      %s" % hex(Rx))
            if Q.y != Ry:
                print("Bad Ry!")
                print("expected %s" % hex(Q.y))
                print("got      %s" % hex(Ry))
        textout = {'Rx': Rx, 'Ry': Ry}        

    else:
        logging.error("Please supply a valid operation to run.")

    cycles = scope.adc.trig_count - start_cycles

    textin = {'operation': operation,
              'Px': Px,
              'Py': Py,
              'k': k
              }
    textout['cycles'] = cycles

    if len(wave) >= 1:
        return Trace(wave, textin, textout, None)
    else:
        return None
